In [23]:
import sys
import os
import logging
import time
import subprocess
from pyhocon import ConfigFactory


# Configure python path to load modules
# Get the current working directory. Should be 'services'
current_dir = os.getcwd()

assert os.path.basename(current_dir) == 'services', 'Current directory is not services'

# Get the parent directory. Should be the root of the repository
parent_dir = os.path.dirname(current_dir)

# The root of the repo should contain the 'software' folder. Otherwise, something went wrong.
assert os.path.exists(os.path.join(parent_dir, 'software')), 'software folder not found in the repository root'

stm_dt_software_dir = os.path.join(parent_dir, 'software')

assert os.path.exists(stm_dt_software_dir), 'stm_dt/software directory not found'

# Add the parent directory to sys.path
sys.path.append(stm_dt_software_dir)

# Import RPCClient class from STM DT, which makes connecting to RabbitMQ and calling remote methods easier.
from communication.rpc_client import RPCClient


In [24]:
# You can run this, or you can run the command below in a new terminal windows to start the service
# Start a process asynchronously
coordinates_anomaly_service_proc = subprocess.Popen([sys.executable, "coordinates_anomaly_service.py"])

# Wait for 5 seconds for the process to start
time.sleep(5)

# Print the PID of the process. You can search for this in your task manager to see the process running and kill it if necessary.
print(coordinates_anomaly_service_proc.pid)

2025-03-08 10:51:26.082 INFO CoordinatesAnomalyService : BusRoutesAnomalyService setup complete.
66116


Traceback (most recent call last):
  File "/Users/carlos_pambo/Projects/Repos/LOG6953FE-STM-DT/services/coordinates_anomaly_service.py", line 80, in <module>
    service.start_serving()
  File "/Users/carlos_pambo/Projects/Repos/LOG6953FE-STM-DT/software/communication/rpc_server.py", line 37, in start_serving
    self.start_consuming()
  File "/Users/carlos_pambo/Projects/Repos/LOG6953FE-STM-DT/software/communication/rabbitmq.py", line 123, in start_consuming
    self.channel.start_consuming()
  File "/opt/anaconda3/lib/python3.12/site-packages/pika/adapters/blocking_connection.py", line 1883, in start_consuming
    self._process_data_events(time_limit=None)
  File "/opt/anaconda3/lib/python3.12/site-packages/pika/adapters/blocking_connection.py", line 2044, in _process_data_events
    self.connection.process_data_events(time_limit=time_limit)
  File "/opt/anaconda3/lib/python3.12/site-packages/pika/adapters/blocking_connection.py", line 842, in process_data_events
    self._flush_outp

In [ ]:
# Print contents of log file to verify that the service is running
with open('CoordinatesAnomalyService.log', 'r') as f:
    print(f.read())

# Check process hasn't crashed
assert coordinates_anomaly_service_proc.poll() is None, "CoordinatesAnomalyService process has crashed"


In [ ]:
# Now that the service is running let us send a request message to execute an operation.

# Get config
startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'stm_dt', 'software', 'config', 'startup.conf')
config = ConfigFactory.parse_file(startup_conf)

with RPCClient(**(config["rabbitmq"])) as client:
    reply = client.invoke_method("coordinates.anomaly.service", "validate_coordinates", {"data": {123: [(46.990, -78.908)]}})
    print(reply)
    assert 'results' in reply, "Reply message does not have key 'results'"
    assert reply["results"].lower() == 'ok', "Reply message key 'results' is not equal to 'ok'"

# Check process hasn't crashed
assert coordinates_anomaly_service_proc.poll() is None, "CoordinatesAnomalyService process has crashed"
